# StanとRでベイズ統計モデリング

詳しい環境とかは、[こちらのmarkdown(Julia_Stan.md)](Julia_Stan.md)

stan.jlを使おうと思ったけど、pystan呼び出して使うことに。。。

In [1]:
# PyCallを経由してpystanのインスタンスを変数pystanにいれる。typeがPyObject
# これでpythonと同じ感覚でモジュールのインスタンスが使えるはず。。
using PyCall
pystan = pyimport("pystan")

PyObject <module 'pystan' from '/usr/local/Caskroom/miniconda/base/envs/conda_jl/lib/python3.8/site-packages/pystan/__init__.py'>

## Stanの基本的な文法


data{<br>データの宣言<br>}<br>parameters{<br>サンプリングしたいパラメータ$\theta$<br>}
<br><br>model{<br>尤度$p(Y|\theta)$<br>事前分布$p(\theta)$<br>}

### memo

- stanでは、値が決まってなく、確率変数とみなせるものは全てparametersにいれる。
- 渡辺ベイズによると、データが確率変数の観測値としてみなせるために、事後分布なんかも確率変数と言うこと。
- 確率モデル、事前分布も自分で定義するもので事後分布も定義してるとみなす。

- 著者によるとモデリングのコツは、
     1. 最初にモデル部分を記述する
     2. それから、dataにデータの変数を記述、残りをparametersに記述

の流れで無理に初めから埋めていかないことがコツらしい。

In [2]:
model = """

data{
    int<lower=0> N; // データの数
    real Y[N];
}

parameters{
    real mu;
}

model{
    for (n in 1:N) {
        Y[n] ~ normal(mu, 1); // normal(mean, std)に注意
    }
    
    mu ~ normal(0, 100);
}

"""

"\ndata{\n    int<lower=0> N; // データの数\n    real Y[N];\n}\n\nparameters{\n    real mu;\n}\n\nmodel{\n    for (n in 1:N) {\n        Y[n] ~ normal(mu, 1); // normal(mean, std)に注意\n    }\n    \n    mu ~ normal(0, 100);\n}\n\n"

- Stanはn内部で対数事後分布を使ってて、MCMC時のパラメータによる$ln\ p(Y|\theta^*)+ln\ p(\theta^*)$を .lp__内に保持している。

- targetという変数を用いて直接定義することもできる

In [3]:
using BenchmarkTools

In [4]:
function fit_sm(model::String)
    data = Dict("N"=>8, "Y"=>[15, 10, 16, 11, 9, 11, 10, 18])
    sm = pystan.StanModel(model_code=model)
    fit = sm.sampling(
            data=data, iter=1000, chains=4, n_jobs=4
    )
    return fit
end

fit_sm (generic function with 1 method)

In [ ]:
# かなり時間がかかる。
@btime res = fit_sm(model)